#  ==================================
# IMPORTS
#  ==================================

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

sns.set_theme()

# jupyter notebook full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# no text wrapping
display(HTML("<style>.dataframe td { white-space: nowrap; }</style>"))

# pandas formatting
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 400)

In [2]:
# import dataframes
df_SD = pd.read_pickle('df_SD.pickle')
df_FD = pd.read_pickle('df_FD.pickle')
df_LF = pd.read_pickle('df_LF.pickle')
df_Site = pd.read_pickle('df_Site.pickle')
df_TrapSupervisors = pd.read_pickle('df_TrapSupervisors.pickle')

#  ==================================
# FISH DETAILS
#  ==================================

## Combine Age 2 and Age 3 Columns

In [3]:
# check if age2 and age3 occur at the same time
(
    df_FD[df_FD['Ager_2'].notnull() & df_FD['Ager_3'].notnull()].shape[0],
    df_FD[df_FD['AGE_2'].notnull() & df_FD['AGE_3'].notnull()].shape[0],
    df_FD[df_FD['FSP_2'].notnull() & df_FD['FSP_3'].notnull()].shape[0], 
    df_FD[df_FD['Comments_2'].notnull() & df_FD['Comments_3'].notnull()].shape[0]
)

(0, 0, 0, 0)

In [5]:
# how many Ager 2 columns
df_FD[df_FD.Ager_2.notnull() | df_FD.AGE_2.notnull() | df_FD.FSP_2.notnull() | df_FD.Comments_2.notnull()].shape[0]

1683

In [4]:
# how many Ager 3 columns
df_FD[df_FD.Ager_3.notnull() | df_FD.AGE_3.notnull() | df_FD.FSP_3.notnull() | df_FD.Comments_3.notnull()].shape[0]

2339

In [6]:
# combine aging 2 and aging 3 into one set of data
df_FD['Ager_2'] = df_FD['Ager_2'].fillna(df_FD['Ager_3'])
df_FD['AGE_2'] = df_FD['AGE_2'].fillna(df_FD['AGE_3'])
df_FD['FSP_2'] = df_FD['FSP_2'].fillna(df_FD['FSP_3'])
df_FD['Comments_2'] = df_FD['Comments_2'].fillna(df_FD['Comments_3'])

In [7]:
# how many Ager 2 columns
df_FD[df_FD.Ager_2.notnull() | df_FD.AGE_2.notnull() | df_FD.FSP_2.notnull() | df_FD.Comments_2.notnull()].shape[0]

4022

## FLAG_SITE: ambiguous site data

In [83]:
# df_FD.loc[df_FD.FLAG_SITE==True]

## FLAG_AM_PM_PERIOD: leave as is

## FLAG_SEX: B and A? -> null

In [45]:
df_FD.loc[df_FD.FLAG_SEX==True, 'SEX'] = np.nan

In [46]:
df_FD.loc[:, 'SEX'].unique()

array(['F', 'M', 'U', nan], dtype=object)

## FLAG_MATURITY: 44=4, 0=null

In [50]:
df_FD.loc[df_FD.MATURITY==44, 'MATURITY'] = 4
df_FD.loc[df_FD.MATURITY==0, 'MATURITY'] = np.nan

## FLAG_FSP_1: 33=3

In [53]:
df_FD.loc[df_FD.FSP_1==33, 'FSP_1'] = 3

## FLAG_FL_STD: 10x off

In [76]:
df_FD.loc[df_FD.FLAG_FL_STD==True, 'FL_STD']

33889   26.100
33890   21.800
33891   26.000
33892   28.300
33893   21.900
         ...  
34470   22.700
34471   23.600
34472   20.800
34473   22.800
34474   20.200
Name: FL_STD, Length: 264, dtype: float64

## FLAG_FL_WET_FROZEN: 5 typos

In [74]:
df_FD.loc[df_FD.FLAG_FL_WET_FROZEN==True, ['FL_WET', 'FL_FROZEN']]

,FL_WET,FL_FROZEN
34215,2,3
34358,28,28
34444,1,1
34445,2,3
34473,2,2


## FLAG_WEIGHT_OUTLIER: 

In [77]:
df_FD.loc[df_FD.FLAG_WEIGHT_OUTLIER==True, 'WEIGHT']

724      519.000
735      539.000
956      501.000
1777     512.000
34178   1934.000
36432    503.000
Name: WEIGHT, dtype: float64

In [105]:
df_FD.loc[df_FD.WEIGHT>1000, 'WEIGHT']

34178   1934.000
Name: WEIGHT, dtype: float64

## FLAG_GONAD_OUTLIER:

In [79]:
df_FD.loc[df_FD.FLAG_GONAD_OUTLIER==True, 'GONAD_WEIGHT']

5604    417.000
13373   766.000
22487   414.800
22723   219.600
25488   280.000
31992   256.000
36765   335.500
Name: GONAD_WEIGHT, dtype: float64

## FLAG_MULTIPLE_SAMPLE_POSSIBILITIES and FLAG_MISNUMBERED_FISH_DETAILS: 

In [81]:
# df_FD.loc[df_FD.FLAG_MULTIPLE_SAMPLE_POSSIBILITIES==True]

## FLAG_LEN_WT_RATIO_OUTLIER: 

In [86]:
# df_FD.loc[df_FD.FLAG_LEN_WT_RATIO_OUTLIER==True, ['FL_STD', 'WEIGHT']]

#  ==================================
# LENGTH FREQUENCIES
#  ==================================

## FLAG_SITE: ambiguous site

In [89]:
# df_LF.loc[df_LF.FLAG_SITE==True]

## FLAG_AM_PM_PERIOD: leave as is

#  ==================================
# SAMPLES
#  ==================================

## FLAG_DATETIME: null datetime

In [90]:
df_SD.loc[df_SD.FLAG_DATETIME==True]

,DIST,RIVER,NAME,code,GEAR,SITE_NO,no_nets,YEAR,MM,DD,Week,catch_lbs,catch_kg,hours_fished,zone,last_name,comments,bycatch_sbass,bycatch_shad,bycatch_other,DATETIME,SITE1,SITE2,remarks,id,total_fish_preserved,total_fish_measured,AM_PM_PERIOD,wt_lbs,FLAG_DATETIME,FLAG_HOURS_FISHED,FLAG_SITE,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_GHOST_SAMPLE
9727,2.000,SWMARG,Darlene Cameron,NaN,81.000,11,1,2006,<NA>,<NA>,<NA>,0.000,0.000,NaN,lower,NaN,NaN,NaN,NaN,NaN,NaT,11,<NA>,NaN,2006000011,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
9728,2.000,SWMARG,John A Chisholm,NaN,81.000,17,1,2006,<NA>,<NA>,<NA>,0.000,0.000,NaN,lower,NaN,NaN,NaN,NaN,NaN,NaT,17,<NA>,NaN,2006000017,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
9729,2.000,SWMARG,Daniel Stewart,NaN,81.000,33,1,2006,<NA>,<NA>,<NA>,0.000,0.000,NaN,upper,NaN,NaN,NaN,NaN,NaN,NaT,33,<NA>,NaN,2006000033,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
9730,2.000,SWMARG,Michael D Gillis,NaN,81.000,48,1,2006,<NA>,<NA>,<NA>,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,48,<NA>,NaN,2006000048,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
9731,2.000,SWMARG,Kevin MacKinnon,NaN,81.000,64,1,2006,<NA>,<NA>,<NA>,0.000,0.000,NaN,NaN,MacKinnon,NaN,NaN,NaN,NaN,NaT,64,<NA>,NaN,2006000064,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
9732,2.000,SWMARG,James MacFarlane,NaN,81.000,65,1,2006,<NA>,<NA>,<NA>,0.000,0.000,NaN,upper,MacFarlane,NaN,NaN,NaN,NaN,NaT,65,<NA>,NaN,2006000065,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
11398,2.000,SWMARG,Charles McDaniel,NaN,81.000,1,1,2010,<NA>,<NA>,<NA>,0.000,0.000,NaN,lower,McDaniel,NaN,NaN,NaN,NaN,NaT,1,<NA>,NaN,2010000001,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
11399,2.000,SWMARG,Michael D Gillis,NaN,81.000,48,1,2010,<NA>,<NA>,<NA>,0.000,0.000,NaN,upper,Gillis,NaN,NaN,NaN,NaN,NaT,48,<NA>,NaN,2010000048,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
11400,2.000,SWMARG,Vincent MacKinnon,NaN,81.000,62,1,2010,<NA>,<NA>,<NA>,0.000,0.000,NaN,upper,MacKinnon,NaN,NaN,NaN,NaN,NaT,62,<NA>,NaN,2010000062,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN
11904,2.000,SWMARG,Joan Ingram,NaN,81.000,34,1,2011,<NA>,<NA>,<NA>,0.000,0.000,0,upper,NaN,NaN,NaN,NaN,NaN,NaT,34,<NA>,NaN,2011000034,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN


## FLAG_HOURS_FISHED: hours_fished = "maximum "

In [92]:
df_SD.loc[df_SD.FLAG_HOURS_FISHED==True, 'hours_fished']

13676    maximum 
13683    maximum 
13691    maximum 
13700    maximum 
13711    maximum 
13722    maximum 
13733    maximum 
13745    maximum 
13758    maximum 
13768    maximum 
13780    maximum 
13791    maximum 
13803    maximum 
13816    maximum 
13829    maximum 
13841    maximum 
13853    maximum 
13866    maximum 
13879    maximum 
13890    maximum 
13902    maximum 
13914    maximum 
13927    maximum 
13940    maximum 
13952    maximum 
13964    maximum 
13975    maximum 
13985    maximum 
Name: hours_fished, dtype: object

## FLAG_SITE: ambiguous site data

In [93]:
# these are currently importing as 1A
df_SD.loc[df_SD.FLAG_SITE==True, 'SITE_NO']

6017    1A,8
6026    1A,8
6035    1A,8
6044    1A,8
6053    1A,8
6062    1A,8
6071    1A,8
6079    1A,8
6087    1A,8
6095    1A,8
6104    1A,8
6113    1A,8
6122    1A,8
6130    1A,8
6139    1A,8
6148    1A,8
6156    1A,8
6165    1A,8
6181    1A,8
6190    1A,8
6199    1A,8
6207    1A,8
6215    1A,8
Name: SITE_NO, dtype: object

## FLAG_AM_PM_PERIOD_DISCREPANCIES: FD and LF inconsistent
#### SAMPLES, LENGTH FREQUENCIES, and FISH DETAILS all flagged
* The samples will be where the AM PM data will be stored, so they are the only updates needed, outside of ghost samples, which can be updated as information becomes available.

In [95]:
df_SD.loc[df_SD.FLAG_AM_PM_PERIOD_DISCREPANCIES==True]

,DIST,RIVER,NAME,code,GEAR,SITE_NO,no_nets,YEAR,MM,DD,Week,catch_lbs,catch_kg,hours_fished,zone,last_name,comments,bycatch_sbass,bycatch_shad,bycatch_other,DATETIME,SITE1,SITE2,remarks,id,total_fish_preserved,total_fish_measured,AM_PM_PERIOD,wt_lbs,FLAG_DATETIME,FLAG_HOURS_FISHED,FLAG_SITE,FLAG_AM_PM_PERIOD_DISCREPANCIES,FLAG_GHOST_SAMPLE
5106,2.000,SWMARG,Daniel Stewart,NaN,81.000,33,NaN,1993,5,13,3,1100.000,499.000,NaN,upper,STEWART,NaN,NaN,NaN,NaN,1993-05-13,33,<NA>,NaN,1993051333,34.000,210.000,AM,115.000,NaN,NaN,NaN,True,NaN
8220,2.000,SWMARG,Pierre Chiasson,NaN,81.000,26,1,2001,6,15,7,500.000,226.800,NaN,lower,Chiasson,NaN,NaN,NaN,NaN,2001-06-15,26,<NA>,NaN,2001061526,51.000,283.000,AM,136.000,NaN,NaN,NaN,True,NaN
10432,2.000,SWMARG,Pierre Chiasson,NaN,81.000,26,1,2007,6,2,5,9600.000,4354.500,8,lower,Chiasson,NaN,NaN,NaN,NaN,2007-06-02,26,<NA>,NaN,2007060226,32.000,290.000,AM,130.000,NaN,NaN,NaN,True,NaN
13507,2.000,SWMARG,Donelda M Gillis,NaN,81.000,47,1,2014,6,10,7,450.000,204.100,7,upper,Gillis,NaN,NaN,NaN,NaN,2014-06-10,47,<NA>,NaN,2014061047,38.000,246.000,AM,89.000,NaN,NaN,NaN,True,NaN


In [97]:
# df_FD.loc[df_FD.FLAG_AM_PM_PERIOD_DISCREPANCIES==True]

In [99]:
# df_LF.loc[df_LF.FLAG_AM_PM_PERIOD_DISCREPANCIES==True]

## FLAG_NO_MATCHING_SAMPLE: no SAMPLE matching LF and/or FD
* these will end up being matched with ghost samples (below)

In [101]:
# df_FD.loc[df_FD.FLAG_NO_MATCHING_SAMPLE==True]

In [103]:
# df_LF.loc[df_LF.FLAG_NO_MATCHING_SAMPLE==True]

## MAKE GHOST SAMPLES
### for unmatched fish details and length frequencies
### NOTE:
Some ambiguous length frequencies and fish details match with eachother. However, we do not know which sample and length frequency are from the same sample. Therefore, we should make sure none of the ambiguous entries ever match automatically, and they are matched manually (eventually) in dm_apps, once that feature is implemented.

Although there are no ambiguous samples that match with length frequencies or fish details, just for throroughness, we should revise id to avoid this possibility as well. 

Therefore, we will arbitrarily add 20 to months in ids of ambiguous fish details, and 40 to months in ids of ambiguous length frequencies. These numbers have no possibility to match a false positive, and they still uniquely identify their date and site.

In [9]:
## to disambiguate ids, add 20 to all months for FD with id>2024000000, add 40 to all months for LF with id>2024000000
df_FD.loc[df_FD.id>2024000000, 'id'] = df_FD.loc[df_FD.id>2024000000, 'id'] + 200000
df_LF.loc[df_LF.id>2024000000, 'id'] = df_LF.loc[df_LF.id>2024000000, 'id'] + 400000

# verified samples are all sample ids before creating ghost samples
verified_samples = set(df_SD.id)

# add null samples where no match exists - Length Frequencies
df_SD = pd.concat([
    df_SD, 
    df_LF[~df_LF.id.isin(verified_samples)].drop_duplicates('id')[['id', 'DATETIME', 'SITE1']]
]).reset_index(drop=True)

# add null samples where no match exists - Fish Details
df_SD = pd.concat([
    df_SD, 
    df_FD[~df_FD.id.isin(verified_samples)].drop_duplicates('id')[['id', 'DATETIME', 'SITE1']]
]).reset_index(drop=True)

# add a Flag to indicate ghost samples for import
df_SD.loc[~df_SD.id.isin(verified_samples), 'FLAG_GHOST_SAMPLE'] = True

# add remarks
df_SD.loc[~df_SD.id.isin(verified_samples), 'remarks'] = 'GHOST SAMPLE, created to match with unmatched Fish Details and/or Length Frequencies'

## REMERGE JOINED DATA
(previously merged without ghost samples, remerge with ghost data)

In [10]:
# drop previously merged columns (to be re-merged)
df_SD = df_SD.drop(['total_fish_preserved', 'total_fish_measured', 'AM_PM_PERIOD', 'wt_lbs'], axis=1)

# JOIN with Fish Details table to get total_fish_preserved
# NOTE: this is an estimate, assuming all fish details are accounted for. This is the best information available
df_SD = pd.merge(
    df_SD, 
    pd.merge(df_SD, df_FD, on='id').groupby('id').count()['FISH_NO'].reset_index(),
    on='id',
    how='left'
).rename({'FISH_NO':'total_fish_preserved'}, axis=1)

# JOIN with Length Frequencies table to get total_fish_measured
# NOTE: this is an estimate, assuming all length frequencies are accounted for. This is the best information available
df_SD = pd.merge(
    df_SD, 
    pd.merge(df_SD, df_LF, on='id').groupby('id').sum()['freq'].reset_index(),
    on='id',
    how='left'
).rename({'freq':'total_fish_measured'}, axis=1)

# # JOIN with Fish Details and Length Frequencies to get AM_PM_PERIOD 
# # NOTE: discrepancies flagged between Length Frequencies and Fish Details
df_period = pd.merge(
    df_FD[df_FD.PERIOD.notnull()].groupby('id').first().reset_index()[['id', 'PERIOD']],
    df_LF[df_LF.period.notnull()].groupby('id').first().reset_index()[['id', 'period']], 
    on='id',
    how='outer'
)
df_period['AM_PM_PERIOD'] = df_period['PERIOD'].fillna(df_period['period'])

df_SD = pd.merge(
    df_SD, 
    df_period[['id', 'AM_PM_PERIOD']],
    on='id',
    how='left'
)

# JOIN with Length Frequency table to get sample weight
df_SD = pd.merge(
    df_SD, 
    df_LF[['id', 'wt_lbs']],
    on='id', 
    how='left'  # all samples 
).drop_duplicates().reset_index(drop=True)

#  ==================================
# LF Grouped
#  ==================================

## Recreate df_LF_grouped with new ambiguous ids for LF entries (+40 to month if ambiguous)

In [17]:
# group by length bins, include only imported columns 
df_LF_grouped = df_LF.groupby(['id', 'length_bin_id']).sum('freq').reset_index()[['freq', 'length_bin_id', 'id']].rename({
    'freq': 'count', 
    'length_bin_id': 'length_bin_id', 
    'id': 'sample_id'
}, axis=1).reset_index(drop=True)[['sample_id', 'length_bin_id', 'count']]

In [71]:
# ambiguous entries, month bit (+40 to disambiguate)
min([int(str(x)[4:6]) for x in list(df_LF_grouped[df_LF_grouped.sample_id>2024000000].sample_id.unique())])

45

In [70]:
# regular entries, month bit
max([int(str(x)[4:6]) for x in list(df_LF_grouped[df_LF_grouped.sample_id<2024000000].sample_id.unique())])

6

#  ==================================
# SAVE DATA
#  ==================================

In [12]:
# reorder columns for clarity - all flags at the end
df_SD = df_SD[[x for x in list(df_SD.columns) if 'FLAG' not in str(x)] + [x for x in list(df_SD.columns) if 'FLAG' in str(x)]]
df_FD = df_FD[[x for x in list(df_FD.columns) if 'FLAG' not in str(x)] + [x for x in list(df_FD.columns) if 'FLAG' in str(x)]]
df_LF = df_LF[[x for x in list(df_LF.columns) if 'FLAG' not in str(x)] + [x for x in list(df_LF.columns) if 'FLAG' in str(x)]]

In [13]:
# save pickle files of dataframes
if save_pickles := False:
    df_SD.to_pickle('df_SD.pickle')
    df_FD.to_pickle('df_FD.pickle')
    df_LF.to_pickle('df_LF.pickle')
    df_LF_grouped.to_pickle('df_LF_grouped.pickle')
    df_Site.to_pickle('df_Site.pickle')
    df_TrapSupervisors.to_pickle('df_TrapSupervisors.pickle')

# save files to this workbook drive
if save_csv := False:
    df_SD.to_csv('gaspereau_sample_data.csv', index=False)
    df_FD.to_csv('gaspereau_fish_details.csv', index=False)
    df_LF.to_csv('gaspereau_length_frequencies.csv', index=False)
    df_Site.to_csv('gaspereau_sites.csv', index=False)
    df_TrapSupervisors.to_csv('gaspereau_trap_supervisors.csv', index=False)
    df_LF_grouped.to_csv('gaspereau_LF_grouped.csv', index=False)
    
# upload to temp folder for import into Kevin's local dm_apps
import_file_location = 'C:\\Users\\CARRK\\Documents\\Repositories\\dm_app_root\\dm_apps\\herring\\temp\\'

if upload_csv_to_temp_folder := False:
    df_SD.to_csv(import_file_location + 'gaspereau_sample_data.csv', index=False)
    df_FD.to_csv(import_file_location + 'gaspereau_fish_details.csv', index=False)
    df_Site.to_csv(import_file_location + 'gaspereau_sites.csv', index=False)
    df_TrapSupervisors.to_csv(import_file_location + 'gaspereau_trap_supervisors.csv', index=False)
    df_LF_grouped.to_csv(import_file_location + 'gaspereau_LF_grouped.csv', index=False)